In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import gc
from keras import backend as K
from keras.layers import BatchNormalization, LayerNormalization, Dense, Embedding, Flatten, Dropout, Reshape, Layer, MultiHeadAttention
from sklearn.model_selection import train_test_split
import keras
from Bio import SeqIO

In [ ]:
print("TensorFlow v" + tf.__version__)
print("Numpy v" + np.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Predefined dictionary
amino_dict = {'0':0, 'A': 1, 'R': 2, 'N': 3, 'D': 4, 'C': 5, 'Q': 6, 'E': 7, 'G': 8, 
              'H': 9, 'I': 10, 'L': 11, 'K': 12, 'M': 13, 'F': 14, 'P': 15, 
              'S': 16, 'T': 17, 'W': 18, 'Y': 19, 'V': 20, 'U':21, 'X':22, 'Z':23, 'O':24, 'B':25, 'Unk':26, '[CLF]':27}

## Pre-processing Functions

In [7]:
def load_sequence(fasta_file, amino_dict, seq_length):
    """
    Load sequences from a FASTA file, convert them to numerical representations, and compute term frequency embeddings.

    Parameters:
    fasta_file (str): Path to the FASTA file containing protein sequences.
    amino_dict (dict): Dictionary mapping amino acids to numerical values.
    seq_length (int): Length to which sequences should be truncated or padded.

    Returns:
    tuple: A tuple containing:
        - vectorized_data (np.ndarray): Array of truncated/padded sequences.
        - ids (np.ndarray): Array of sequence IDs.
        - org_sequences (list): List of original sequences.
        - lengths (np.ndarray): Array of original sequence lengths.
        - tf_embeddings (np.ndarray): Array of term frequency embeddings.
    """
    
    print('File to process:', fasta_file)
    
    # Initialize lists to store data
    list_ids, org_sequences, trunc_sequences, lengths, c, tf_embeddings = [], [], [], [], 0, []
    
    # Parse the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        
        list_ids.append(record.id)  # Store sequence ID
        
        seq = np.array(list(record.seq))  # Convert sequence to numpy array

        selected_tokens = np.zeros(seq_length)  # Initialize array for truncated/padded sequence

        tf_rep = [1] * len(amino_dict.keys())  # Initialize term frequency representation

        if seq.shape[0] >= seq_length:
            # If sequence is longer than or equal to the desired length, truncate it
            for i in range(seq_length):
                tf_rep[amino_dict[seq[i]]] += 1  # Update term frequency
                if seq[i] in amino_dict.keys():
                    selected_tokens[i] = amino_dict.get(seq[i])  # Convert amino acid to numerical value
                else:
                    selected_tokens[i] = 25  # Use id 25 for unknown amino acids
        else:
            # If sequence is shorter than the desired length, pad it
            for i in range(len(seq)):
                tf_rep[amino_dict[seq[i]]] += 1  # Update term frequency
                if seq[i] in amino_dict.keys():
                    selected_tokens[i] = amino_dict.get(seq[i])  # Convert amino acid to numerical value
                else:
                    selected_tokens[i] = 25  # Use id 25 for unknown amino acids

        tf_embeddings.append(tf_rep)  # Store term frequency representation
        trunc_sequences.append(selected_tokens)  # Store truncated/padded sequence
        org_sequences.append(seq)  # Store original sequence
        lengths.append(len(record.seq))  # Store original sequence length
        
        c += 1
        print(f'{c:.0f} proteins processed', end='\r')  # Print progress
    print()
        
    # Convert lists to numpy arrays
    vectorized_data = np.array(trunc_sequences)
    tf_embeddings = np.log(np.array(tf_embeddings))
    ids = np.array(list_ids)
    lengths = np.array(lengths)

    # Sort data by sequence IDs
    sorting_indexes = np.argsort(ids)
    vectorized_data = vectorized_data[sorting_indexes]
    tf_embeddings = tf_embeddings[sorting_indexes]
    lengths = lengths[sorting_indexes]
    ids = ids[sorting_indexes]

    return vectorized_data, ids, org_sequences, lengths, tf_embeddings

def load_labels(tsv_term_file, score_text_file, list_ids, num_labels_f, min_score):
    """
    Load labels for protein sequences based on term frequency and score.

    Parameters:
    tsv_term_file (str): Path to the TSV file containing terms.
    score_text_file (str): Path to the text file containing scores.
    list_ids (list): List of protein sequence IDs.
    num_labels_f (int): Number of most frequent terms to consider.
    min_score (float): Minimum score threshold for terms.

    Returns:
    tuple: A tuple containing:
        - label_matrix (np.ndarray): Matrix of labels for protein sequences.
        - label_list (list): List of terms that meet the criteria.
    """
    
    series_train_protein_ids = pd.Series(list_ids)  # Convert list of IDs to a pandas Series
    
    train_terms = pd.read_csv(tsv_term_file, sep="\t")  # Load terms from TSV file

    label_list_f = train_terms['term'].value_counts().index[:num_labels_f].tolist()  # Get most frequent terms

    score_data = np.genfromtxt(score_text_file, dtype=None, encoding=None)  # Load scores from text file
    scores = np.zeros(score_data.shape[0])  # Initialize array for scores

    for i in range(score_data.shape[0]):
        scores[i] = score_data[i][1]  # Populate scores array
    
    indexes = np.where(scores > min_score)[0]  # Get indexes of scores above the minimum score

    label_list_s = score_data[indexes]  # Get terms with scores above the minimum score

    label_list = []  # Initialize list for final terms

    for j in range(len(label_list_f)):
        for k in range(label_list_s.shape[0]):
            if label_list_f[j] == label_list_s[k][0]:
                label_list.append(label_list_f[j])  # Add term to final list if it meets the criteria

    num_labels = len(label_list)  # Get the number of final terms

    print(f'The number of terms with a minimum score of {min_score} that are also in the most {num_labels_f} frequent terms is {num_labels}')

    label_matrix = np.zeros((len(list_ids), num_labels))  # Initialize label matrix
    
    for i in range(num_labels):
        
        n_train_terms = train_terms[train_terms['term'] == label_list[i]]  # Get terms matching the final list
        
        label_related_proteins = n_train_terms['EntryID'].unique()  # Get unique protein IDs for the term

        label_matrix[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)  # Populate label matrix
        
        p = i/num_labels * 100  # Calculate progress percentage
        
        print(f'{p:.0f}% completed', end='\r')  # Print progress
    
    return label_matrix, label_list  # Return label matrix and final list of terms

def load_protein_embd(embd_file, ids_file, del_duplicate=False):
    """
    Load protein embeddings and their corresponding IDs from specified files.

    Args:
        embd_file (str): Path to the file containing the protein embeddings.
        ids_file (str): Path to the file containing the protein IDs.
        del_duplicate (bool, optional): If True, deletes the duplicate entry at index 632. Defaults to False.

    Returns:
        tuple: A tuple containing:
            - ids (numpy.ndarray): Sorted array of protein IDs.
            - embeddings (numpy.ndarray): Sorted array of protein embeddings corresponding to the IDs.
    """
    embeddings = np.load(embd_file)
    ids = np.load(ids_file)
    sorting_indexes = np.argsort(ids)
    embeddings = embeddings[sorting_indexes]
    ids = ids[sorting_indexes]

    if del_duplicate:
        embeddings = np.delete(embeddings, 632, 0)
        ids = np.delete(ids, 632, 0)

    return ids, embeddings

## Load and Preprocess data

In [8]:
LEN_AMINO_SEQ = 300
NUM_FREQ_TERMS = 550
THRESHOLD_VAL = 0.01

In [ ]:
# Load training sequences and their term frequency embeddings
train_seq_tokens, train_ids, train_org_seq, train_org_seq_lengths, tf_embd_train = load_sequence(
    "Train/train_sequences.fasta", amino_dict, LEN_AMINO_SEQ)

# Load test sequences and their term frequency embeddings
test_seq_tokens, test_ids, test_org_seq, test_org_seq_lengths, tf_embd_test = load_sequence(
    "Test (Targets)/testsuperset.fasta", amino_dict, LEN_AMINO_SEQ)

# Remove the weird entry in the test set
test_seq_tokens = np.delete(test_seq_tokens, 632, 0)
tf_embd_test = np.delete(tf_embd_test, 632, 0)
test_org_seq_lengths = np.delete(test_org_seq_lengths, 632, 0)
test_ids = np.delete(test_ids, 632, 0)
test_org_seq.pop(632)

# Load protein embeddings and their corresponding IDs for training and test sets
ids_train_embeddings, train_embeddings = load_protein_embd('ems2/train_embeddings.npy', 'ems2/train_ids.npy')
ids_test_embeddings, test_embeddings = load_protein_embd('ems2/test_embeddings.npy', 'ems2/test_ids.npy')

# Load labels for the training set
train_labels, label_list = load_labels("Train/train_terms.tsv", 'IA.txt', train_ids, NUM_FREQ_TERMS, THRESHOLD_VAL)

# Create classification tokens for training and test sets
clf_tokens_train = np.ones((tf_embd_train.shape[0], 1)) * 27
clf_tokens_test = np.ones((tf_embd_test.shape[0], 1)) * 27

# Combine embeddings, term frequency embeddings, sequence tokens, and classification tokens into a single matrix for training and test sets
train_matrix = np.hstack((np.hstack((np.hstack((train_embeddings, tf_embd_train)), train_seq_tokens)), clf_tokens_train))
test_matrix = np.hstack((np.hstack((np.hstack((test_embeddings, tf_embd_test)), test_seq_tokens)), clf_tokens_test))

## Transformer BERT-type Model

In [14]:
class FeedForwardMLP(Layer):
    """
    A simple feed-forward multi-layer perceptron (MLP) with two dense layers and dropout.
    """

    def __init__(self, n_embd, dropout, **kwargs):
        """
        Initialize the FeedForwardMLP layer.

        Args:
            n_embd (int): The embedding dimension.
            dropout (float): Dropout rate.
            **kwargs: Additional keyword arguments for the Layer base class.
        """
        super().__init__(**kwargs)
        self.net = tf.keras.Sequential([
            Dense(4 * n_embd, activation='relu'),  # First dense layer with ReLU activation
            Dense(n_embd),  # Second dense layer
            Dropout(dropout)  # Dropout layer
        ])

    def call(self, inputs):
        """
        Forward pass of the FeedForwardMLP.

        Args:
            inputs (Tensor): Input tensor.

        Returns:
            Tensor: Output tensor after passing through the MLP.
        """
        return self.net(inputs)


class Block(Layer):
    """
    A transformer block consisting of multi-head self-attention and feed-forward layers.
    """

    def __init__(self, n_embd, n_head, dropout, **kwargs):
        """
        Initialize the Block layer.

        Args:
            n_embd (int): The embedding dimension.
            n_head (int): Number of attention heads.
            dropout (float): Dropout rate.
            **kwargs: Additional keyword arguments for the Layer base class.
        """
        super().__init__(**kwargs)

        head_size = n_embd // n_head

        self.sa = MultiHeadAttention(n_head, key_dim=n_embd)  # Multi-head self-attention layer
        self.fw = FeedForwardMLP(n_embd, dropout)  # Feed-forward MLP layer

        self.ln1 = LayerNormalization()  # Layer normalization before self-attention
        self.ln2 = LayerNormalization()  # Layer normalization before feed-forward MLP

    def call(self, x):
        """
        Forward pass of the Block.

        Args:
            x (Tensor): Input tensor.

        Returns:
            Tensor: Output tensor after passing through the block.
        """
        q = self.ln1(x)
        k, v = q, q

        x = x + self.sa(q, k, v)  # Add and normalize after self-attention
        x = x + self.fw(self.ln2(x))  # Add and normalize after feed-forward MLP

        return x


class Network(keras.Model):
    """
    A custom neural network model consisting of embedding layers, transformer blocks, and a final output layer.
    """

    def __init__(self, seq_length, n_heads, dict_length, embd_dim, num_blocks, dropout, out_dim, prot_embd_dim, **kwargs):
        """
        Initialize the Network model.

        Args:
            seq_length (int): Sequence length.
            n_heads (int): Number of attention heads.
            dict_length (int): Length of the dictionary (vocabulary size).
            embd_dim (int): Embedding dimension.
            num_blocks (int): Number of transformer blocks.
            dropout (float): Dropout rate.
            out_dim (int): Output dimension.
            prot_embd_dim (int): Protein embedding dimension.
            **kwargs: Additional keyword arguments for the Model base class.
        """
        super().__init__(**kwargs)

        self.seq_length = seq_length
        self.dict_length = dict_length
        self.embd = prot_embd_dim

        self.protein_embd = Dense(units=embd_dim, activation='relu')  # Protein embedding layer
        self.tf_embd = Dense(units=embd_dim, activation='relu')  # TF embedding layer

        self.token_embd = Embedding(input_dim=dict_length, output_dim=embd_dim, input_length=seq_length + 1)  # Token embedding layer
        self.pos_embd = Embedding(input_dim=seq_length + 3, output_dim=embd_dim, input_length=seq_length + 3)  # Positional embedding layer
        self.segment_embd = Embedding(input_dim=4, output_dim=embd_dim, input_length=seq_length + 5)  # Segment embedding layer

        self.blocks = [Block(embd_dim, n_heads, dropout) for i in range(num_blocks)]  # List of transformer blocks

        self.out = Dense(out_dim, "sigmoid")  # Output layer

    def call(self, X):
        """
        Forward pass of the Network model.

        Args:
            X (Tensor): Input tensor.

        Returns:
            Tensor: Output tensor after passing through the network.
        """

        # Split the input tensor into protein tokens, TF tokens, and sequence tokens
        cut_1 = self.embd
        cut_2 = cut_1 + self.dict_length
    
        # X_1 = Protein tokens, X_2 = TF tokens, X_seq = Sequence tokens
        X_1, X_2, X_seq = X[:, 0:cut_1], X[:, cut_1:cut_2], X[:, cut_2:]
        # Cast X_seq to integer type
        X_seq = tf.cast(X_seq, tf.int32)
        # Create a list of positional indices from 0 to seq_length + 3 (3 extra tokens: protein, TF, and CLF)
        pos_list = np.arange(self.seq_length + 3)

        # Create a list of segment indices (0 for CLF, 1 for protein, 2 for TF, 3 for sequence)
        segments = np.ones(self.seq_length + 3)
        segments[-1] = 0
        segments[0:cut_1] = 1
        segments[cut_1:cut_2] = 2
        segments[cut_2:-1] = 3

        # Embed the segment indices
        segment_embd = self.segment_embd(segments)

        # Embed the protein tokens
        prot_token = self.protein_embd(X_1)
        prot_token = prot_1_token[:, tf.newaxis, :]

        # Embed the TF tokens
        tok_tf_embd = self.tf_embd(X_2)
        tok_tf_embd = tok_tf_embd[:, tf.newaxis, :]

        # Embed the sequence tokens
        tok_seq_embd = self.token_embd(X_seq)
        pos_embd = self.pos_embd(pos_list)

        # Concatenate the embeddings
        tokens = tf.concat([prot_token, tok_tf_embd, tok_seq_embd], axis=1)

        # Add the positional and segment embeddings
        Z = tokens + pos_embd + segment_embd

        # Pass the embeddings through the transformer blocks
        for block in self.blocks:
            Z = block(Z)

        # Extract the CLF token
        CLF_token = Z[:, -1, :]
        # Pass the CLF token through the output layer
        CL = self.out(CLF_token)

        return CL


# Model parameters
sequence_length = train_seq_tokens.shape[1]
protein_embedding_dim = train_embeddings.shape[1]
output_dim = train_labels.shape[1]
num_encoder_blocks = 10
num_att_heads = 4
dropout_rate = 0.2
hidden_embedding_dim = 120

model = Network(seq_length=sequence_length, 
                n_heads=num_att_heads, 
                dict_length=len(amino_dict.keys()), 
                embd_dim=prot, 
                num_blocks=num_encoder_blocks, 
                dropout=dropout_rate, 
                out_dim=output_dim, 
                prot_embd_dim=protein_embedding_dim)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', tf.keras.metrics.AUC()],
)

In [ ]:
history = model.fit(train_matrix, train_labels, epochs=4, validation_split=0.1, batch_size=256)

# Plot the model's loss and accuracy for each epoch

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy']].plot(title="Accuracy")

# Submission

In [ ]:
del train_matrix
del train_ids
del tf_embd_train
del ids_train_embeddings
del train_embeddings
del train_seq_tokens
del train_org_seq
del train_org_seq_lengths
K.clear_session()
gc.collect()

#test_matrix = np.random.randint(low=0, high=len(amino_dict.keys()), size=(141865,1700))
predictions =  model.predict(test_matrix)

In [ ]:
def submission(predictions, test_ids, label_list):
    
    df_submission = pd.DataFrame(columns = ['Protein Id', 'GO Term Id','Prediction'])
    l = []
    for k in test_ids:
        l += [ k] * predictions.shape[1]   

    df_submission['Protein Id'] = l
    df_submission['GO Term Id'] = label_list * predictions.shape[0]
    df_submission['Prediction'] = predictions.ravel()
    df_submission.to_csv("submission.tsv",header=False, index=False, sep="\t")
    
    return df_submission
    

In [ ]:
del model
del test_matrix
del tf_embd_test
del ids_test_embeddings
del test_embeddings
del test_seq_tokens
del test_org_seq
del test_org_seq_lengths
gc.collect()

df_submission = submission(predictions, test_ids, label_list)